<a href="https://colab.research.google.com/github/steveregis/eat-right/blob/main/EatBuddy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#EatBuddy

Random pilot to test if we can create a intelligent chatbot that can provide personalized advise on allergies found in a given menu


In [ ]:
!pip install transformers==4.35.0 accelerate bitsandbytes
!pip install huggingface_hub>=0.14.1
!apt-get update && apt-get install -y g++ cmake libssl-dev
!pip install auto-gptq
!pip install transformers accelerate

In [ ]:
!pip install --upgrade huggingface_hub accelerate

In [ ]:
from transformers import AutoTokenizer, pipeline
from auto_gptq import AutoGPTQForCausalLM  # Correct GPTQ loader

model_id = "TheBloke/Llama-2-13B-chat-GPTQ"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

# Load the GPTQ model
model = AutoGPTQForCausalLM.from_quantized(model_id, device="cuda:0", use_safetensors=True)

# Create a text-generation pipeline
llama_pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

# Test the pipeline
result = llama_pipe("Hello, how are you?")
print(result)

In [ ]:
def generate(prompt, max_length=6144, pipe=llama_pipe, **kwargs):
    """
def generate(prompt, max_length=4096, pipe=llama_pipe, **kwargs):

    Generates a response to the given prompt using a specified language model pipeline.

    This function takes a prompt and passes it to a language model pipeline, such as LLaMA,
    to generate a text response. The function is designed to allow customization of the
    generation process through various parameters and keyword arguments.

    Parameters:
    - prompt (str): The input text prompt to generate a response for.
    - max_length (int): The maximum length of the generated response. Default is 1024 tokens.
    - pipe (callable): The language model pipeline function used for generation. Default is llama_pipe.
    - **kwargs: Additional keyword arguments that are passed to the pipeline function.

    Returns:
    - str: The generated text response from the model, trimmed of leading and trailing whitespace.

    Example usage:
    ```
    prompt_text = "Explain the theory of relativity."
    response = generate(prompt_text, max_length=512, pipe=my_custom_pipeline, temperature=0.7)
    print(response)
    ```
    """

    def_kwargs = dict(return_full_text=False, return_dict=False)
    response = pipe(prompt.strip(), max_length=max_length, **kwargs, **def_kwargs)
    return response[0]['generated_text'].strip()

In [ ]:
def construct_prompt_with_context(main_prompt, system_context="", conversation_examples=[]):
    """
    Constructs a complete structured prompt for a language model, including optional system context and conversation examples.

    This function compiles a prompt that can be directly used for generating responses from a language model.
    It creates a structured format that begins with an optional system context message, appends a series of conversational
    examples as prior interactions, and ends with the main user prompt. If no system context or conversation examples are provided,
    it will return only the main prompt.

    Parameters:
    - main_prompt (str): The core question or statement for the language model to respond to.
    - system_context (str, optional): Additional context or information about the scenario or environment. Defaults to an empty string.
    - conversation_examples (list of tuples, optional): Prior exchanges provided as context, where each tuple contains a user message
      and a corresponding agent response. Defaults to an empty list.

    Returns:
    - str: A string formatted as a complete prompt ready for language model input. If no system context or examples are provided, returns the main prompt.

    Example usage:
    ```
    main_prompt = "I'm looking to improve my dialogue writing skills for my next short story. Any suggestions?"
    system_context = "User is an aspiring author seeking to enhance dialogue writing techniques."
    conversation_examples = [
        ("How can dialogue contribute to character development?", "Dialogue should reveal character traits and show personal growth over the story arc."),
        ("What are some common pitfalls in writing dialogue?", "Avoid exposition dumps in dialogue and make sure each character's voice is distinct.")
    ]

    full_prompt = construct_prompt_with_context(main_prompt, system_context, conversation_examples)
    print(full_prompt)
    ```
    """

    # Return the main prompt if no system context or conversation examples are provided
    if not system_context and not conversation_examples:
        return main_prompt

    # Start with the initial part of the prompt including the system context, if provided
    full_prompt = f"<s>[INST] <<SYS>>{system_context}<</SYS>>\n" if system_context else "<s>[INST]\n"

    # Add each example from the conversation_examples to the prompt
    for user_msg, agent_response in conversation_examples:
        full_prompt += f"{user_msg} [/INST] {agent_response} </s><s>[INST]"

    # Add the main user prompt at the end
    full_prompt += f"{main_prompt} [/INST]"

    return full_prompt

In [ ]:
def print_token_count(text, tokenizer):
    """
    Calculate and return the number of tokens in a given text using a specified tokenizer.

    This function takes a string of text and a tokenizer. It uses the tokenizer to encode the text
    into tokens and then returns the count of these tokens.

    Parameters:
    - text (str): The input string to be tokenized.
    - tokenizer: A tokenizer instance capable of encoding text into tokens.

    Returns:
    - int: The number of tokens in the input text as determined by the tokenizer.
    """
    return len(tokenizer.encode(text))

In [ ]:
def concat_history(tuples_list):
    """
    Concatenates texts from a list of 2-tuples.

    Each tuple in the list is expected to contain two strings. The function
    will concatenate all the first elements followed by all the second elements
    in their respective order of appearance in the list.

    Parameters:
    - tuples_list (list of 2-tuples): A list where each element is a tuple of two strings.

    Returns:
    - str: A single string that is the result of concatenating all the texts from the tuples.

    Example usage:
    ```
    conversation_tuples = [
        ('Question 1', 'Answer 1'),
        ('Question 2', 'Answer 2'),
        ('Question 3', 'Answer 3')
    ]

    concatenated_text = concatenate_texts_from_tuples(conversation_tuples)
    print(concatenated_text)
    ```
    """
    # Concatenate all the first and second elements of the tuples
    return ''.join(question + response for question, response in tuples_list)

In [ ]:
menu= [
    {
      "name": "Hummus",
      "ingredients": ["Chickpeas", "Tahini", "Olive oil", "Lemon juice", "Garlic", "Cumin", "Salt"],
      "allergies": {
        "Sesame": {
          "related_ingredient": "Tahini",
          "possible_symptoms": ["Hives", "Swelling", "Difficulty breathing", "Gastrointestinal issues"],
          "caution": "Avoid if you have a sesame allergy."
        }
      }
    },
    {
      "name": "Falafel",
      "ingredients": ["Chickpeas", "Onion", "Garlic", "Parsley", "Cumin", "Coriander", "Salt", "Pepper", "Flour", "Baking powder", "Oil for frying"],
      "allergies": {
        "Gluten": {
          "related_ingredient": "Flour",
          "possible_symptoms": ["Bloating", "Diarrhea", "Fatigue", "Headaches"],
          "caution": "Avoid if you have celiac disease or gluten intolerance."
        }
      }
    },
    {
      "name": "Tabbouleh",
      "ingredients": ["Parsley", "Tomatoes", "Onion", "Bulgur", "Mint", "Lemon juice", "Olive oil", "Salt"],
      "allergies": {
        "Gluten": {
          "related_ingredient": "Bulgur",
          "possible_symptoms": ["Bloating", "Diarrhea", "Fatigue", "Headaches"],
          "caution": "Avoid if you have celiac disease or gluten intolerance."
        }
      }
    },
    {
      "name": "Shawarma",
      "ingredients": ["Marinated meat (chicken, beef, or lamb)", "Garlic", "Yogurt", "Lemon juice", "Spices (cumin, paprika, etc.)", "Pita bread", "Vegetables (tomato, cucumber, onion)", "Tahini or garlic sauce"],
      "allergies": {
        "Dairy": {
          "related_ingredient": "Yogurt, Tahini",
          "possible_symptoms": ["Hives", "Nausea", "Abdominal pain", "Anaphylaxis in severe cases"],
          "caution": "Avoid if you have a dairy allergy or lactose intolerance."
        },
        "Gluten": {
          "related_ingredient": "Pita bread",
          "possible_symptoms": ["Bloating", "Diarrhea", "Fatigue", "Headaches"],
          "caution": "Avoid if you have celiac disease or gluten intolerance."
        }
      }
    },
    {
      "name": "Fattoush",
      "ingredients": ["Lettuce", "Tomatoes", "Cucumbers", "Radishes", "Green onions", "Pita bread", "Sumac", "Olive oil", "Lemon juice"],
      "allergies": {
        "Gluten": {
          "related_ingredient": "Pita bread",
          "possible_symptoms": ["Bloating", "Diarrhea", "Fatigue", "Headaches"],
          "caution": "Avoid if you have celiac disease or gluten intolerance."
        }
      }
    },
    {
      "name": "Baba Ghanoush",
      "ingredients": ["Eggplant", "Tahini", "Olive oil", "Lemon juice", "Garlic", "Salt"],
      "allergies": {
        "Sesame": {
          "related_ingredient": "Tahini",
          "possible_symptoms": ["Hives", "Swelling", "Difficulty breathing", "Gastrointestinal issues"],
          "caution": "Avoid if you have a sesame allergy."
        }
      }
    },
    {
      "name": "Kebabs",
      "ingredients": ["Ground meat (beef, lamb, or chicken)", "Onion", "Garlic", "Spices (cumin, coriander, paprika)", "Salt", "Pepper"],
      "allergies": {}
    },
    {
      "name": "Mujadara",
      "ingredients": ["Lentils", "Rice", "Onions", "Olive oil", "Cumin", "Salt", "Pepper"],
      "allergies": {}
    },
    {
      "name": "Dolma",
      "ingredients": ["Grape leaves", "Rice", "Ground meat", "Onion", "Tomato paste", "Spices (cinnamon, allspice)", "Lemon juice"],
      "allergies": {}
    },
    {
      "name": "Manakish",
      "ingredients": ["Flatbread", "Za'atar", "Olive oil", "Cheese", "Meat (optional)"],
      "allergies": {
        "Gluten": {
          "related_ingredient": "Flatbread",
          "possible_symptoms": ["Bloating", "Diarrhea", "Fatigue", "Headaches"],
          "caution": "Avoid if you have celiac disease or gluten intolerance."
        },
        "Dairy": {
          "related_ingredient": "Cheese",
          "possible_symptoms": ["Hives", "Nausea", "Abdominal pain", "Anaphylaxis in severe cases"],
          "caution": "Avoid if you have a dairy allergy or lactose intolerance."
        }
      }
    },
    {
      "name": "Kofta",
      "ingredients": ["Ground meat (beef or lamb)", "Onion", "Garlic", "Parsley", "Spices (cumin, cinnamon)", "Salt", "Pepper"],
      "allergies": {}
    },
    {
      "name": "Sfeeha",
      "ingredients": ["Ground meat", "Onions", "Tomato", "Pine nuts", "Spices (cinnamon, allspice)", "Dough"],
      "allergies": {
        "Nuts": {
          "related_ingredient": "Pine nuts",
          "possible_symptoms": ["Hives", "Swelling", "Nausea", "Anaphylaxis in severe cases"],
          "caution": "Avoid if you have a nut allergy."
        },
        "Gluten": {
          "related_ingredient": "Dough",
          "possible_symptoms": ["Bloating", "Diarrhea", "Fatigue", "Headaches"],
          "caution": "Avoid if you have celiac disease or gluten intolerance."
        }
      }
    },
    {
      "name": "Labneh",
      "ingredients": ["Yogurt", "Salt", "Olive oil", "Za'atar (optional)"],
      "allergies": {
        "Dairy": {
          "related_ingredient": "Yogurt",
          "possible_symptoms": ["Hives", "Nausea", "Abdominal pain", "Anaphylaxis in severe cases"],
          "caution": "Avoid if you have a dairy allergy or lactose intolerance."
        },
        "Sesame": {
          "related_ingredient": "Za'atar",
          "possible_symptoms": ["Hives", "Swelling", "Difficulty breathing", "Gastrointestinal issues"],
          "caution": "Avoid if you have a sesame allergy."
        }
      }
    },
    {
      "name": "Baklava",
      "ingredients": ["Phyllo pastry", "Nuts (walnuts, pistachios)", "Honey", "Sugar", "Butter", "Cinnamon"],
      "allergies": {
        "Nuts": {
          "related_ingredient": "Walnuts, Pistachios",
          "possible_symptoms": ["Hives", "Swelling", "Nausea", "Anaphylaxis in severe cases"],
          "caution": "Avoid if you have a nut allergy."
        },
        "Gluten": {
          "related_ingredient": "Phyllo pastry",
          "possible_symptoms": ["Bloating", "Diarrhea", "Fatigue", "Headaches"],
          "caution": "Avoid if you have celiac disease or gluten intolerance."
        },
        "Dairy": {
          "related_ingredient": "Butter",
          "possible_symptoms": ["Hives", "Nausea", "Abdominal pain", "Anaphylaxis in severe cases"],
          "caution": "Avoid if you have a dairy allergy or lactose intolerance."
        }
      }
    },
    {
      "name": "Qatayef",
      "ingredients": ["Flour", "Yeast", "Sugar", "Milk", "Nuts or cheese filling", "Syrup"],
      "allergies": {
        "Gluten": {
          "related_ingredient": "Flour",
          "possible_symptoms": ["Bloating", "Diarrhea", "Fatigue", "Headaches"],
          "caution": "Avoid if you have celiac disease or gluten intolerance."
        },
        "Dairy": {
          "related_ingredient": "Milk, Cheese filling",
          "possible_symptoms": ["Hives", "Nausea", "Abdominal pain", "Anaphylaxis in severe cases"],
          "caution": "Avoid if you have a dairy allergy or lactose intolerance."
        },
        "Nuts": {
          "related_ingredient": "Nuts (if filled)",
          "possible_symptoms": ["Hives", "Swelling", "Nausea", "Anaphylaxis in severe cases"],
          "caution": "Avoid if you have a nut allergy."
        }
      }
    },
    {
      "name": "Harissa",
      "ingredients": ["Chili peppers", "Garlic", "Cumin", "Coriander", "Olive oil"],
      "allergies": {}
    },
    {
      "name": "Biryani",
      "ingredients": ["Rice", "Chicken or lamb", "Spices (saffron, cardamom)", "Onions", "Yogurt", "Mint", "Cilantro"],
      "allergies": {
        "Dairy": {
          "related_ingredient": "Yogurt",
          "possible_symptoms": ["Hives", "Nausea", "Abdominal pain", "Anaphylaxis in severe cases"],
          "caution": "Avoid if you have a dairy allergy or lactose intolerance."
        }
      }
    },
    {
      "name": "Khobz",
      "ingredients": ["Flour", "Water", "Yeast", "Salt"],
      "allergies": {
        "Gluten": {
          "related_ingredient": "Flour",
          "possible_symptoms": ["Bloating", "Diarrhea", "Fatigue", "Headaches"],
          "caution": "Avoid if you have celiac disease or gluten intolerance."
        }
      }
    },
    {
      "name": "Kousa Mahshi",
      "ingredients": ["Zucchini", "Rice", "Ground meat", "Tomato sauce", "Spices", "Onion"],
      "allergies": {}
    },
    {
      "name": "Bamya",
      "ingredients": ["Okra", "Tomatoes", "Onion", "Garlic", "Lamb or beef", "Spices", "Olive oil"],
      "allergies": {}
    },
    {
      "name": "Feseekh",
      "ingredients": ["Fermented fish", "Lemon", "Onion", "Salad"],
      "allergies": {
        "Fish": {
          "related_ingredient": "Fermented fish",
          "possible_symptoms": ["Hives", "Nausea", "Abdominal pain", "Anaphylaxis in severe cases"],
          "caution": "Avoid if you have a fish allergy."
        }
      }
    },
    {
      "name": "Sambousek",
      "ingredients": ["Dough", "Ground meat or cheese", "Onions", "Spices", "Oil for frying"],
      "allergies": {
        "Gluten": {
          "related_ingredient": "Dough",
          "possible_symptoms": ["Bloating", "Diarrhea", "Fatigue", "Headaches"],
          "caution": "Avoid if you have celiac disease or gluten intolerance."
        },
        "Dairy": {
          "related_ingredient": "Cheese",
          "possible_symptoms": ["Hives", "Nausea", "Abdominal pain", "Anaphylaxis in severe cases"],
          "caution": "Avoid if you have a dairy allergy or lactose intolerance."
        }
      }
    },
    {
      "name": "Zaatar Manakish",
      "ingredients": ["Flatbread", "Zaatar spice mix", "Olive oil"],
      "allergies": {
        "Gluten": {
          "related_ingredient": "Flatbread",
          "possible_symptoms": ["Bloating", "Diarrhea", "Fatigue", "Headaches"],
          "caution": "Avoid if you have celiac disease or gluten intolerance."
        },
        "Sesame": {
          "related_ingredient": "Zaatar",
          "possible_symptoms": ["Hives", "Swelling", "Difficulty breathing", "Gastrointestinal issues"],
          "caution": "Avoid if you have a sesame allergy."
        }
      }
    },
    {
      "name": "Mansaf",
      "ingredients": ["Lamb", "Rice", "Jameed (dried yogurt)", "Pine nuts", "Almonds", "Spices"],
      "allergies": {
        "Dairy": {
          "related_ingredient": "Jameed",
          "possible_symptoms": ["Hives", "Nausea", "Abdominal pain", "Anaphylaxis in severe cases"],
          "caution": "Avoid if you have a dairy allergy."
        },
        "Nuts": {
          "related_ingredient": "Pine nuts, Almonds",
          "possible_symptoms": ["Hives", "Swelling", "Nausea", "Anaphylaxis in severe cases"],
          "caution": "Avoid if you have a nut allergy."
        }
      }
    },
    {
      "name": "Kousa Bil-Lahme",
      "ingredients": ["Zucchini", "Ground beef", "Tomato sauce", "Onion", "Spices", "Rice"],
      "allergies": {}
    },
    {
      "name": "Tunisian Brik",
      "ingredients": ["Brik pastry", "Tuna", "Capers", "Egg", "Parsley", "Lemon"],
      "allergies": {
        "Fish": {
          "related_ingredient": "Tuna",
          "possible_symptoms": ["Hives", "Nausea", "Abdominal pain", "Anaphylaxis in severe cases"],
          "caution": "Avoid if you have a fish allergy."
        },
        "Egg": {
          "related_ingredient": "Egg",
          "possible_symptoms": ["Hives", "Nausea", "Abdominal pain", "Anaphylaxis in severe cases"],
          "caution": "Avoid if you have an egg allergy."
        }
      }
    },
    {
      "name": "Maqlooba",
      "ingredients": ["Rice", "Chicken or lamb", "Eggplant", "Cauliflower", "Spices", "Tomato sauce"],
      "allergies": {}
    }
  ]


In [ ]:
class LlamaChatbot:
    """
    A chatbot interface for generating conversational responses using the LLaMA language model.

    Attributes:
    - system_context (str): Contextual information to provide to the language model for all conversations.
    - conversation_history (list of tuples): Stores the history of the conversation, where each
      tuple contains a user message and the corresponding agent response.
    """

    def __init__(self, system_context):
        """
        Initializes a new instance of the LlamaChatbot class.

        Parameters:
        - system_context (str): A string that sets the initial context for the language model.
        """
        self.system_context = system_context
        self.conversation_history = []  # Initializes the conversation history

    def chat(self, user_msg):
        """
        Generates a response from the chatbot based on the user's message.

        This method constructs a prompt with the current system context and conversation history,
        sends it to the language model, and then stores the new user message and model's response
        in the conversation history.

        Parameters:
        - user_msg (str): The user's message to which the chatbot will respond.

        Returns:
        - str: The generated response from the chatbot.
        """
        # Generate the prompt using the conversation history and the new user message
        prompt = construct_prompt_with_context(user_msg, self.system_context, self.conversation_history)

        # Get the model's response
        agent_response = generate(prompt)

        # Store this interaction in the conversation history
        self.conversation_history.append((user_msg, agent_response))

        return agent_response

    def reset(self):
        """
        Resets the conversation history of the chatbot.

        This method clears the existing conversation history, effectively restarting the conversation.
        """
        # Clear conversation history
        self.conversation_history = []

In [ ]:
class LlamaChatbotWithHistoryLimit:
    """
    A chatbot interface for generating conversational responses using the LLaMA language model.

    Attributes:
        - system_context (str): Contextual information to provide to the language model for all conversations.
        - conversation_history (list of tuples): Stores the history of the conversation, where each
          tuple contains a user message and the corresponding agent response.
        - tokenizer: The tokenizer used to tokenize the conversation for maintaining the history limit.
        - max_tokens (int): The maximum number of tokens allowed in the conversation history.
    """

    def __init__(self, system_context, tokenizer, max_tokens=2048):
        """
        Initializes a new instance of the LlamaChatbot class with a tokenizer and token limit.

        Parameters:
            - system_context (str): A string that sets the initial context for the language model.
            - tokenizer: The tokenizer used to process the input and output for the language model.
            - max_tokens (int): The maximum number of tokens to retain in the conversation history.
        """
        self.system_context = system_context
        self.tokenizer = tokenizer
        self.max_tokens = max_tokens
        self.conversation_history = []  # Initializes the conversation history

    def chat(self, user_msg):
        """
        Generates a response from the chatbot based on the user's message.

        This method constructs a prompt with the current system context and conversation history,
        sends it to the language model, and then stores the new user message and model's response
        in the conversation history, ensuring that the history does not exceed the specified token limit.

        Parameters:
            - user_msg (str): The user's message to which the chatbot will respond.

        Returns:
            - str: The generated response from the chatbot.
        """
        # Generate the prompt using the conversation history and the new user message
        prompt = construct_prompt_with_context(user_msg, self.system_context, self.conversation_history)

        # Get the model's response
        agent_response = generate(prompt)

        # Store this interaction in the conversation history
        self.conversation_history.append((user_msg, agent_response))

        # Check and maintain the conversation history within the token limit
        self._trim_conversation_history()

        return agent_response

    def _trim_conversation_history(self):
        """
        Trims the conversation history to maintain the number of tokens below the specified limit.
        """
        # Concatenate the conversation history into a single string
        history_string = ''.join(user + agent for user, agent in self.conversation_history)

        # Calculate the number of tokens in the conversation history
        history_tokens = len(self.tokenizer.encode(history_string))

        # While the history exceeds the maximum token limit, remove the oldest items
        while history_tokens > self.max_tokens:
            # Always check if there's at least one item to pop
            if self.conversation_history:
                # Remove the oldest conversation tuple
                self.conversation_history.pop(0)
                # Recalculate the history string and its tokens
                history_string = ''.join(user + agent for user, agent in self.conversation_history)
                history_tokens = len(self.tokenizer.encode(history_string))
            else:
                # If the conversation history is empty, break out of the loop
                break

    def reset(self):
        """
        Resets the conversation history of the chatbot.

        This method clears the existing conversation history, effectively restarting the conversation.
        """
        # Clear conversation history
        self.conversation_history = []

In [ ]:
system_context = f"""
You are a friendly chatbot knowledgeable about food , its ingredients and allergies {menu}. \
When asked about specific menu or ingredient, you try to provide accurate and helpful answers. \
Your goal is to assist and inform potential customers to the best of your ability in 50 words or less. \
You always end by asking what else you can help with.
"""

chatbot = LlamaChatbotWithHistoryLimit(system_context, tokenizer=tokenizer, max_tokens=200)

In [ ]:
system_context = f"""
You are a friendly chatbot knowledgeable about food , its ingredients and allergies {menu}. \
When asked about specific menu or ingredient, you try to provide accurate and helpful answers. \
Your goal is to assist and inform potential customers to the best of your ability in 50 words or less. \
You always end by asking what else you can help with.
"""

chatbot = LlamaChatbot(system_context)

In [ ]:
print(chatbot.chat(" latest allergies in the menu?"))

In [ ]:
print(chatbot.chat("i have high sugar can u suggest a item?"))

In [ ]:
print_token_count(concat_history(chatbot.conversation_history), tokenizer)

In [ ]:
print(chatbot.chat("can i get a Baklava?"))

In [ ]:
print(chatbot.chat("last week i had something with seasame which made me sick you know , i dont want get sick again"))

In [ ]:
print(chatbot.chat("im craving for a shawarma"))

In [ ]:
print(chatbot.chat("and may be Zaatar Manakish followed by Baklava "))


In [ ]:
print(chatbot.chat("Can you remind me of my allergies"))


In [ ]:
chatbot.reset()